In [44]:
%%configure -f
{"executorCores": 4, "executorMemory": "32768M", "conf": {"spark.default.parallelism": 1000}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1567158732142_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1567158732142_0007,pyspark,idle,Link,Link,✔


In [45]:
import pandas as pd

import math
import gensim
import random
from gensim.utils import tokenize
from gensim.parsing.preprocessing import remove_stopwords
import string
import spacy
from langdetect import detect
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, ArrayType, DoubleType, MapType, BooleanType
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.sql.window import Window

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df = spark.read.load("s3://ai-data-lake-dev-eu-west-1/staging/peer/company_denormalized_extra_field")
df.count()
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- one_id: string (nullable = true)
 |-- company_id: integer (nullable = true)
 |-- company_name: string (nullable = true)
 |-- street_address: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_code: string (nullable = true)
 |-- company_type: string (nullable = true)
 |-- number_of_employees: integer (nullable = true)
 |-- latest_revenue: double (nullable = true)
 |-- latest_revenue_growth: double (nullable = true)
 |-- latest_ebitda: double (nullable = true)
 |-- latest_ebitda_margin: double (nullable = true)
 |-- competitor_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- simple_industry_id: string (nullable = true)
 |-- simple_industry: string (nullable = true)
 |-- sic_code: string (nullable = true)
 |-- sic_code_desc: string (nullable = true)
 |-- naics_code: string (nullable = true)
 |-- naics_code_des

In [47]:
df_filtered = (df.filter((F.length('country') > 0) & 
                         (F.length('region') > 0) & 
                         (F.length('company_description') > 10))
              )
# df_filtered.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df_competitor = df_filtered.filter(F.size('competitor_ids') > 0)
df_competitor.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

45756

In [49]:
df_positive = df_competitor.withColumn('competitor_company_id', F.explode('competitor_ids'))
df_positive.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

706152

In [50]:
df_competitor_renamed = df_filtered.toDF(*('competitor_'+c for c in df.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
df_positive = df_positive.join(df_competitor_renamed, 'competitor_company_id')
df_positive = df_positive.withColumn("label", F.lit(1))
df_positive.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

664511

In [52]:
df_positive_filtered_pre=(df_positive.filter(
                                         (F.col('simple_industry_id')==F.col('competitor_simple_industry_id'))
                                        ))
df_positive_filtered_pre.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

383071

In [53]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

jaccard_similarity_udf = udf(jaccard_similarity, DoubleType())

df_positive_filtered_pre = df_positive_filtered_pre.withColumn("jaccard_location", jaccard_similarity_udf("operating_locations", "competitor_operating_locations"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
df_positive_filtered=(df_positive_filtered_pre.filter(
                                         (F.col('jaccard_location')>0) 
                                        & (F.col('latest_revenue') > 0) & (F.col('competitor_latest_revenue') > 0)
                                        )).drop('jaccard_location')
df_positive_filtered.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

88681

In [55]:
company_ids = (df_filtered.filter(F.col('company_type')=="Public Company").select('company_id', 'simple_industry_id'))
company_ids.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

61781

In [56]:
company_ids = company_ids.withColumn("new_column",F.lit("ABC"))
w = Window().partitionBy('new_column').orderBy(F.lit('A'))
company_ids = company_ids.withColumn("idx", F.row_number().over(w)).drop("new_column").withColumnRenamed('company_id','competitor_company_id')
nb_company = company_ids.count()
# nb_company

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
industry_id_map = company_ids.groupBy('simple_industry_id').agg(F.collect_list("idx").alias("per_industry_ids"))
industry_id_map.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- simple_industry_id: string (nullable = true)
 |-- per_industry_ids: array (nullable = true)
 |    |-- element: integer (containsNull = true)

In [58]:
def get_random(nb_company, k=25):
    return random.sample(range(1,1+nb_company), k)

get_random_udf = udf(get_random, ArrayType(IntegerType()))

def get_random_from_index(index, k=25):
    if len(index) < k:
        return []
    idx = random.sample(range(0, len(index)), k)
    return [index[i] for i in idx]

get_random_from_index_udf = udf(get_random_from_index, ArrayType(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
company_ids = company_ids.drop('simple_industry_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
df_competitor_negatives = (df_competitor
 .join(industry_id_map, 'simple_industry_id')
 .withColumn('idx_all', get_random_udf(F.lit(nb_company)))
 .withColumn('idx_same_industry', get_random_from_index_udf('per_industry_ids'))
 .withColumn('idx', F.concat(F.col('idx_all'), F.col('idx_same_industry')))
 .drop('idx_all', 'idx_same_industry', 'per_industry_ids')
 .withColumn('idx', F.explode('idx'))
 .join(company_ids, 'idx')
 .drop('idx')
)
# df_competitor_negatives.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
df_negative = df_competitor_negatives.join(df_competitor_renamed, 'competitor_company_id')
df_negative = df_negative.withColumn("label", F.lit(0))
# df_negative.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
def filter_negative(negative_company_id, positive_company_ids):
    if negative_company_id in positive_company_ids:
        return False
    return True

filter_negative_udf = udf(filter_negative, BooleanType())
df_negative_filtered = df_negative.filter(filter_negative_udf("competitor_company_id", "competitor_ids"))
# df_negative_filtered.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
df_negative_filtered = df_negative_filtered.select(list(df_positive_filtered.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
df_all = df_positive_filtered.union(df_negative_filtered).withColumn("idx", F.monotonically_increasing_id())
df_all = df_all.cache()
# df_all.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
nlp = spacy.load("en_core_web_sm")

def process_text(text):
    doc = nlp(text)
    words = []
    count = 0
    for sent in doc.sents:
        if "was founded in" in sent.string or "a subsidiary of" in sent.string:
            continue
        for word in sent:
            if word.is_stop or word.is_punct or not word.is_alpha or len(word.text) <= 1:
                continue
            words.append(word.lemma_.lower())
    return words

process_text_udf = udf(process_text, ArrayType(StringType()))

df_preprocess = (df_filtered.withColumn("tokenized_text", process_text_udf("company_description"))
                 .withColumn("idx", F.monotonically_increasing_id())
                 .withColumn('token', F.explode('tokenized_text'))
                )

df_tokens = (df_preprocess.groupBy("token")
             .agg(F.countDistinct("idx").alias('df'))
            )

nb_docs = df_filtered.count()
nb_docs

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3343215

In [66]:
def calcIdf(doc_count, df):
    return math.log((doc_count+1)/(df+1))

calIdf_udf = udf(calcIdf, DoubleType())

df_idf = df_tokens.withColumn("idf", calIdf_udf(F.lit(nb_docs), 'df'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
# (df_idf
#  .write
#  .parquet("s3://ai-data-lake-dev-eu-west-1/staging/peer/idf", mode="overwrite")
# )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
# from io import StringIO
# import boto3

df_idf = df_idf.select('token', 'idf')

# csv_buffer = StringIO()
# df_idf.toPandas().to_csv(csv_buffer)
# s3_resource = boto3.resource('s3')
# s3_resource.Object('ai-data-lake-dev-eu-west-1', 'staging/peer/idf.csv').put(Body=csv_buffer.getvalue())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
df_idf = df_idf.cache();

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
# df_idf = spark.read.format("csv").option("header", "true").load("s3://ai-data-lake-dev-eu-west-1/staging/peer/idf.csv").drop("_c0")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
df_idf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- token: string (nullable = true)
 |-- idf: double (nullable = true)

In [72]:
df_all = (df_all.withColumn("tokenized_text", process_text_udf("company_description"))
          .withColumn("description_length", F.size("tokenized_text"))
          .withColumn("competitor_tokenized_text", process_text_udf("competitor_company_description"))
          .withColumn("competitor_description_length", F.size("competitor_tokenized_text"))
         )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
df_all_preprocess = df_all.withColumn('token', F.explode('tokenized_text'))
df_all_preprocess_tf = df_all_preprocess.groupBy("idx", "token").agg(F.count("tokenized_text").alias("tf"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
combineMap = udf(lambda maps: {key:f[key] for f in maps for key in f}, MapType(StringType(),ArrayType(DoubleType())))    

df_all_preprocess_tf_idf = (df_all_preprocess_tf
      .join(df_idf, "token", "left")
      .withColumn("tf_idf", F.array("tf","idf"))
      .withColumn("tf_idf_token", F.create_map("token","tf_idf"))
      .groupBy("idx")
      .agg(F.collect_list("tf_idf_token").alias("tf_idf_text_list"))
      .withColumn("tf_idf_map", combineMap("tf_idf_text_list"))
      .drop("tf_idf_text_list", "idf", "tf", "token")
)

df_all_preprocess_tf_idf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- idx: long (nullable = false)
 |-- tf_idf_map: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: double (containsNull = true)

In [75]:
df_train = df_all.join(df_all_preprocess_tf_idf, "idx")
# df_train.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
df_all_negative_preprocess = df_all.withColumn('token', F.explode('competitor_tokenized_text'))
df_all_preprocess_competitor_tf = df_all_negative_preprocess.groupBy("idx", "token").agg(F.count("competitor_tokenized_text").alias("tf"))
df_all_preprocess_competitor_tf.printSchema()
df_idf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- idx: long (nullable = false)
 |-- token: string (nullable = true)
 |-- tf: long (nullable = false)

root
 |-- token: string (nullable = true)
 |-- idf: double (nullable = true)

In [77]:
df_all_preprocess_competitor_tf_idf = (df_all_preprocess_competitor_tf
      .join(df_idf, "token", "left")
      .withColumn("tf_idf", F.array("tf","idf")) #F.col("tf") * F.col("idf"))
      .withColumn("tf_idf_token", F.create_map("token","tf_idf"))
      .groupBy("idx")
      .agg(F.collect_list("tf_idf_token").alias("tf_idf_text_list"))
      .withColumn("competitor_tf_idf_map", combineMap("tf_idf_text_list"))
      .drop("tf_idf_text_list", "idf", "tf", "competitor_token")
)

df_all_preprocess_competitor_tf_idf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- idx: long (nullable = false)
 |-- competitor_tf_idf_map: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: double (containsNull = true)

In [78]:
df_train = df_train.join(df_all_preprocess_competitor_tf_idf, "idx").drop('tokenized_text', 'competitor_tokenized_text')
# df_train.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
df_train.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- idx: long (nullable = false)
 |-- competitor_company_id: string (nullable = true)
 |-- one_id: string (nullable = true)
 |-- company_id: integer (nullable = true)
 |-- company_name: string (nullable = true)
 |-- street_address: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_code: string (nullable = true)
 |-- company_type: string (nullable = true)
 |-- number_of_employees: integer (nullable = true)
 |-- latest_revenue: double (nullable = true)
 |-- latest_revenue_growth: double (nullable = true)
 |-- latest_ebitda: double (nullable = true)
 |-- latest_ebitda_margin: double (nullable = true)
 |-- competitor_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- simple_industry_id: string (nullable = true)
 |-- simple_industry: string (nullable = true)
 |-- sic_code: string (nullable = true)
 |-- sic_code_desc:

In [80]:
def dot(A,B, use_tf=False):
    total = 0
    for key, val in A.items(): # val[0] = tf, val[1] = idf
        if key in B and val[1] and B[key][1]:
            tmp = val[1]*B[key][1]
            if use_tf:
                tmp *= math.log(val[0]+1)*math.log(B[key][0]+1)
            total += tmp
    return total

def compute_cosine_similarity(tf_idf_a, tf_idf_b, use_tf=False):
    if not tf_idf_a or not tf_idf_b:
        return 0
    return dot(tf_idf_a,tf_idf_b, use_tf) / ( (dot(tf_idf_a,tf_idf_a, use_tf) **.5) * (dot(tf_idf_b,tf_idf_b, use_tf) ** .5) )

compute_cosine_similarity_udf = udf(compute_cosine_similarity, DoubleType())

df_train = df_train.withColumn("cosine_similarity", compute_cosine_similarity_udf('tf_idf_map', 'competitor_tf_idf_map', F.lit(False)))
df_train = df_train.withColumn("cosine_similarity_tf", compute_cosine_similarity_udf('tf_idf_map', 'competitor_tf_idf_map', F.lit(True)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
match_same_field_udf = udf(lambda x,y: int(x == y))

df_train = df_train.withColumn("match_country", match_same_field_udf('country', 'competitor_country'))
df_train = df_train.withColumn("match_region", match_same_field_udf('region', 'competitor_region'))
df_train = df_train.withColumn("match_company_type", match_same_field_udf('company_type', 'competitor_company_type'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
df_train = df_train.withColumn("jaccard_location", jaccard_similarity_udf("operating_locations", "competitor_operating_locations"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
def numerical_distance(v1, v2):
    if not v1 or not v2 or v1 == 0 or v2 == 0:
        return 0

    if v1 < v2:
        return v1/v2
    else:
        return v2/v1

numerical_distance_udf = udf(numerical_distance, DoubleType())

df_train = df_train.withColumn("revenue_distance", numerical_distance_udf("latest_revenue", "competitor_latest_revenue"))
df_train = df_train.withColumn("ebitda_distance", numerical_distance_udf("latest_ebitda", "competitor_latest_ebitda"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
df_train.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- idx: long (nullable = false)
 |-- competitor_company_id: string (nullable = true)
 |-- one_id: string (nullable = true)
 |-- company_id: integer (nullable = true)
 |-- company_name: string (nullable = true)
 |-- street_address: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_code: string (nullable = true)
 |-- company_type: string (nullable = true)
 |-- number_of_employees: integer (nullable = true)
 |-- latest_revenue: double (nullable = true)
 |-- latest_revenue_growth: double (nullable = true)
 |-- latest_ebitda: double (nullable = true)
 |-- latest_ebitda_margin: double (nullable = true)
 |-- competitor_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- simple_industry_id: string (nullable = true)
 |-- simple_industry: string (nullable = true)
 |-- sic_code: string (nullable = true)
 |-- sic_code_desc:

In [85]:
# features = (df_train.select(['idx', 'company_id','competitor_company_id', 'cosine_similarity', 'cosine_similarity_tf',
#                             'match_country', 'match_region', 'match_company_type', 
#                             'description_length', 'competitor_description_length',
#                             'jaccard_location', 'revenue_distance', 'ebitda_distance', 'label']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [86]:
(df_train
 .write
 .parquet("s3://ai-data-lake-dev-eu-west-1/staging/peer/features_sic_code", mode="overwrite")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_train = spark.read.load("s3://ai-data-lake-dev-eu-west-1/staging/peer/features_sic_code")

In [ ]:
def sic_code_distance(sic_code1, sic_code2):
    return abs(sic_code1 - sic_code2)

sic_code_distance_udf = udf(sic_code_distance, IntegerType())


df_train = df_train.withColumn("sic_code_distance", sic_code_id_distance_udf("sic_code", "competitor_sic_code"))